In [1]:
import pandas as pd
import pandas_profiling

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df_train  = pd.read_csv('train.csv') # reading training dataset

In [3]:
df_train.head()

,cloth_comfort,username,size,product,weight,score,purpose,shape_of_body,product_type,review,date,age,cloth_size
0,fit,44334,34d,2260466,137lbs,10.0,vacation,hourglass,romper,An adorable romper! Belt and zipper were a lit...,2016-04-20,28.0,14
1,fit,28835,34b,153475,132lbs,10.0,other,straight & narrow,gown,I rented this dress for a photo shoot. The the...,2013-06-18,36.0,12
2,fit,37976,NaN,1063761,NaN,10.0,party,NaN,sheath,This hugged in all the right places! It was a ...,2015-12-14,116.0,4
3,fit,96080,34c,126335,135lbs,8.0,formal affair,pear,dress,I rented this for my company's black tie award...,2014-02-12,34.0,8
4,fit,15959,34b,616682,145lbs,10.0,wedding,athletic,gown,I have always been petite in my upper body and...,2016-09-26,27.0,12


In [4]:
df_test = pd.read_csv('test.csv')

In [5]:
df_test.head()

,Unnamed: 0,username
0,0,69
1,1,125
2,2,148
3,3,173
4,4,239


In [6]:
temp = pd.merge(df_train,df_test,on='username',how='inner') # getting number of user which are present in both train and test file

In [7]:
len(list(set(temp['username'])))# only 1422 is present out of 3179 users, So remaining are new users

1422

In [8]:
# !pip install pandas==0.25
profile = pandas_profiling.ProfileReport(df_train) # visualizing data
profile.to_file('profiler.html')


/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [9]:
profile

Number of variables,13
Number of observations,154033
Total Missing (%),2.5%
Total size in memory,15.3 MiB
Average record size in memory,104.0 B
Numeric,5
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [10]:
# filling missing values
import numpy as np
df_train['size'].replace(np.nan, 'unknown', inplace=True)
df_train['weight'].replace(np.nan,'unknown',inplace=True)
df_train['shape_of_body'].replace(np.nan,'unknown',inplace=True)
df_train['product_type'].replace(np.nan,'unknown',inplace=True)
df_train['review'].replace(np.nan,'unknown',inplace=True)
df_train['age'].replace(np.nan,0,inplace=True)
df_train['purpose'].replace(np.nan,'unknown',inplace=True)
df_train['score'].replace(np.nan,0,inplace=True)

In [11]:
from textblob import TextBlob
from tqdm import tqdm
polarity = []
for i in tqdm(df_train['review']):
    polarity.append(TextBlob(i).sentiment.polarity)
#     break
df_train['polarity'] = polarity
df_train.drop('review',axis=1,inplace=True)

100%|██████████| 154033/154033 [02:09<00:00, 1193.04it/s]


In [12]:
import numpy as np 
from sklearn.model_selection import train_test_split
df_train_org = df_train.copy()

In [13]:
# Creating user features
uf = []
col = ['size']*len(df_train_org['size'].unique())+ ['weight']*len(df_train_org['weight'].unique()) + ['shape_of_body']*len(df_train_org['shape_of_body'].unique()) + ['age']*len(df_train_org['age'].unique())
unique_f1 = list(df_train_org['size'].unique()) + list(df_train_org['weight'].unique()) + list(df_train_org['shape_of_body'].unique()) + list(df_train_org['age'].unique())
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    uf.append(res)
    

In [15]:
#creating item features
if1 = []
col = ['cloth_comfort']*len(df_train_org['cloth_comfort'].unique()) +  ['product_type']*len(df_train_org['product_type'].unique()) 
unique_f1 = list(df_train_org['cloth_comfort'].unique()) + list(df_train_org['product_type'].unique())
#print('f1:', unique_f1)
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    if1.append(res)


In [16]:
# !pip install lightfm
from lightfm.data import Dataset
# we call fit to supply userid, item id and user/item features
dataset1 = Dataset()
dataset1.fit(
        df_train_org['username'].unique(), # all the users
        df_train_org['product'].unique(), # all the items
        #user_features = ['f1:1', 'f1:0', 'f2:1', 'f2:0', 'f3:1', 'f3:0']
        user_features = uf,# additional user feature
        item_features = if1# additional item feature
)


In [17]:
# plugging in the interactions and their weights
(interactions, weights) = dataset1.build_interactions([(int(x[0]), x[1], x[2]) for x in df_train_org[['username','product','score']].values ])

In [18]:
def feature_colon_value(my_list,col):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    #  = ['f1:','f2:', 'f3:', 'loc:']
    aa = my_list
    for x,y in zip(col,aa):
        res = str(x) +":"+ str(y)
        result.append(res)
    return result


In [19]:
# creating dataset for lightfm
ad_subset = df_train_org[['cloth_comfort','product_type']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item,['cloth_comfort','product_type']))


In [20]:
ad_subset = df_train_org[['size','weight','shape_of_body','age']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list_user = []
for item in ad_list:
    feature_list_user.append(feature_colon_value(item,['size','weight','shape_of_body','age']))


In [21]:
item_tuple = list(zip(df_train_org['product'], feature_list))


In [22]:
user_tuple = list(zip(df_train_org['username'], feature_list_user))


In [23]:
user_features = dataset1.build_user_features(user_tuple, normalize= False)


In [24]:
item_features = dataset1.build_item_features(item_tuple, normalize= False)

In [26]:
# creating mapping for item/user to there ids
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()
user_id_map = pd.DataFrame.from_dict(user_id_map,orient='index').reset_index()
user_feature_map = pd.DataFrame.from_dict(user_feature_map,orient='index').reset_index()
item_id_map = pd.DataFrame.from_dict(item_id_map,orient='index').reset_index()
item_feature_map = pd.DataFrame.from_dict(item_feature_map,orient='index').reset_index()

In [27]:
# !pip install lightfm
from lightfm import LightFM

model = LightFM(loss='warp',no_components=100,max_sampled=250,epsilon=1e-1,learning_schedule='adadelta')

In [28]:
model.fit(interactions,# spase matrix representing whether user u and item i interacted
          user_features=user_features,# we have built the sparse matrix above
          # item_features=item_features, # item features was not useful when it comes to predicting the recommendation
          sample_weight = weights,# spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
          epochs=20,verbose=1,num_threads=4)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [29]:
from lightfm.evaluation import recall_at_k,precision_at_k
tr_precision = precision_at_k(model, interactions,user_features=user_features,k=5,num_threads=4).mean()
tr_recall = recall_at_k(model, interactions,user_features=user_features,
           k=5,num_threads=4).mean()

In [30]:
print (tr_precision,tr_recall)

0.23892431 0.8545018405299424


In [31]:
# getting top 5 recommendations
recommendation_user = []
from tqdm import tqdm
for i in tqdm(range(len(user_id_map))):
  user_x = user_id_map.loc[i,0]
  n_users, n_items = interactions.shape # no of users * no of items
  scores = list(model.predict(user_x,np.arange(n_items)))
  scores_df = pd.DataFrame()
  scores_df['score'] = scores
  scores_df['product'] = item_id_map['index']
  # print (item_id_map)
  nrecommendation = scores_df.nlargest(6, 'score')['product'].tolist()[1:]
  recommendation_user.append(nrecommendation)

100%|██████████| 89502/89502 [11:17<00:00, 132.05it/s]


In [32]:
recommendation = pd.DataFrame(recommendation_user)
recommendation.columns=['recommendation_'+str(i+1) for i in recommendation.columns if i>=0]
user_id_map = pd.concat([user_id_map,recommendation],axis=1)

In [35]:
user_id_map.drop(0,axis=1,inplace=True)
user_id_map.head()

,index,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
0,44334,1083818,2431951,2340996,1257812,1133906
1,28835,145906,154002,127865,125564,131698
2,37976,1697200,985499,195076,148690,1260731
3,96080,174086,145906,136110,131117,131533
4,15959,616682,1427750,524341,1710886,2309132


In [49]:
df_test.head()

,Unnamed: 0,username
0,0,69
1,1,125
2,2,148
3,3,173
4,4,239


In [37]:
test = df_test.merge(user_id_map,left_on='username',right_on='index',how='left')

In [38]:
df_train_org['score']

0         10.0
1         10.0
2         10.0
3          8.0
4         10.0
          ... 
154028     8.0
154029    10.0
154030    10.0
154031     6.0
154032    10.0
Name: score, Length: 154033, dtype: float64

In [40]:
highly_rated = df_train_org[df_train_org['score']>6]# getting popular product which are going to recommended for new users with no search history

In [41]:
top_rated_products = list(pd.DataFrame((highly_rated['product']).value_counts()).index[0:5])

In [42]:
top_rated_products

[126335, 174086, 123793, 132738, 145906]

In [43]:
test.head()

,Unnamed: 0,username,index,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
0,0,69,NaN,NaN,NaN,NaN,NaN,NaN
1,1,125,NaN,NaN,NaN,NaN,NaN,NaN
2,2,148,148.0,1875147.0,1701960.0,2463317.0,131533.0,172027.0
3,3,173,173.0,1141067.0,2275436.0,2827079.0,2097490.0,1850708.0
4,4,239,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
for i in range(len(test)):
  for j in range(5):
    if str(test.loc[i,'recommendation_'+str(j+1)])=='nan':
      test.at[i,'recommendation_'+str(j+1)] = top_rated_products[j]

In [45]:
test.head()

,Unnamed: 0,username,index,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
0,0,69,NaN,126335.0,174086.0,123793.0,132738.0,145906.0
1,1,125,NaN,126335.0,174086.0,123793.0,132738.0,145906.0
2,2,148,148.0,1875147.0,1701960.0,2463317.0,131533.0,172027.0
3,3,173,173.0,1141067.0,2275436.0,2827079.0,2097490.0,1850708.0
4,4,239,NaN,126335.0,174086.0,123793.0,132738.0,145906.0


In [46]:
test.drop(['index','Unnamed: 0'],axis=1,inplace=True)

In [47]:
test.head()

,username,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
0,69,126335.0,174086.0,123793.0,132738.0,145906.0
1,125,126335.0,174086.0,123793.0,132738.0,145906.0
2,148,1875147.0,1701960.0,2463317.0,131533.0,172027.0
3,173,1141067.0,2275436.0,2827079.0,2097490.0,1850708.0
4,239,126335.0,174086.0,123793.0,132738.0,145906.0


In [48]:
test.to_csv('./drive/My Drive/dataleap/result.csv',index=False)